In [10]:
import torch
import torch.nn as nn
from torchvision.models import resnet34
from datasets import MelSpecDataset

import seaborn as sns

In [11]:
model = resnet34()

In [31]:
#def main():
BATCH_SIZE = 2
TEST_SIZE = 0.2

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

dataset = MelSpecDataset(csv_loc="./data/wav_files.csv", data_dir="./data")

# train-test split
test_index = int(len(dataset)*TEST_SIZE)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-test_index]) # first 20%
dataset_test = torch.utils.data.Subset(dataset, indices[-test_index:]) # last 80%

# define data loader
data_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
data_loader = torch.utils.data.DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# get model
model = resnet34()
model.to(device)

# construct optimiser
parameters = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

# let's train it for 10 epochs
num_epochs = 10

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

